# Final Project

In [1]:
import os
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow
from tqdm import tqdm
import tensorflow.keras as keras
import frame_extractor as extractor
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

2023-11-18 20:17:34.692544: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-18 20:17:35.572544: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_X = []
train_Y = []
parent = os.path.dirname(os.getcwd())
videos = os.listdir(parent + "/train_sample_videos/")
f = open(parent + "/train_sample_videos/metadata.json")
valid = json.load(f)
for video in tqdm(videos):
    if video != "metadata.json":
        for frame in extractor.run_extraction(parent + "/train_sample_videos/" + video):
            train_X.append(frame)
            break
        train_Y.append(1 if valid[video]["label"] == "FAKE" else 0)

100%|██████████| 401/401 [00:46<00:00,  8.64it/s]


In [3]:
def check_orientation(train_X):
    h_sum = 0
    v_sum = 0
    uhoh_sum = 0
    for nar in train_X:
        if(nar.shape==(1920,1080,3)):
            h_sum+=1
        elif(nar.shape==(1080,1920,3)):
            v_sum+=1
        else:
            uhoh_sum+=1


    print(h_sum)
    print(v_sum)
    print(uhoh_sum)

In [4]:
def rotate_vertical(x):
    if x.shape == (1920,1080,3):
        return np.rot90(x)
    else:
        return x

In [5]:
rotated_train_X = list(map(rotate_vertical, train_X))

In [6]:
train_X = rotated_train_X

In [7]:
#Ignore this for now, I don't think we want to filter verticals, 
#but maybe in the future we'll have to, so this code is nice as a failsafe
# def filter_horizontal(x):
#     return x.shape==(1080,1920,3)
# only_horizontal = list(filter(filter_horizontal, train_X))
# check_orientation(only_horizontal)


# Creating the Model
This cell creates the cnn model to be trained. (Currently a basic CNN example model, not our finished model)

In [18]:
cnn_epochs = 10
cnn_batch_size = 16
train_samples = len(train_X)#TODO: Input number of samples: something like "len(x_train)", if x_train is our training array
input_shape = (1080,1920,3) #TODO: Set input_shape to the shape of our input
num_classes = 2

steps_per_epoch = train_samples/cnn_batch_size

cnn_model = tensorflow.keras.models.Sequential()
cnn_model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape)) 
cnn_model.add(Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.25))
cnn_model.add(Flatten())
# cnn_model.add(Dense(32, activation='relu'))
cnn_model.add(Dense(num_classes, activation='softmax'))
cnn_model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(),
              metrics=['accuracy'])

In [17]:
cnn_model.fit(x=np.array(train_X),y=np.array(train_Y),epochs=cnn_epochs)

Epoch 1/10


2023-11-18 20:24:33.885580: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:1014] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_6/dropout_5/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-11-18 20:24:44.147489: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 759.38MiB (rounded to 796262400)requested by op sequential_6/Cast
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-11-18 20:24:44.147582: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-11-18 20:24:44.147611: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 75, Chunks in use: 74. 18.8KiB allocated for chunks. 18.5KiB in use in bin. 3.2KiB client-requested in use 

ResourceExhaustedError: Graph execution error:

Detected at node 'sequential_6/Cast' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.10/asyncio/base_events.py", line 600, in run_forever
      self._run_once()
    File "/usr/lib/python3.10/asyncio/base_